In [12]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
import cv2
from pycocotools.coco import COCO
from PIL import Image
import torch
import os
import matplotlib.pyplot as plt
#import colormap as cmap
import numpy as np


def visualize(index, masks, bounding_boxes):
    image = coco.loadImgs(coco.getImgIds()[index])[0]    
    anno_ids = coco.getAnnIds(imgIds=[image["id"]])
    annos = coco.loadAnns(anno_ids) 
    
    """new_annos = []
    for single in annos:
        single = get_angles(single)
        new_annos.append([single])
    
    annos = new_annos"""
    filename = os.path.join(data_folder,image["file_name"])
    img = cv2.imread(str(filename))
    print(img.shape)
    for anno in annos:
        #print(anno)
        anno = get_angles(anno)
        #c = tuple([int(x) for x in anno["category_id"]])         
        if masks:
            points = np.array(anno["segmentation"], dtype=np.int32).reshape(1,-1,2)
            img = cv2.polylines(img, points, True, 3)
        
        if bounding_boxes:  
                x,y,w,h,a = np.array(anno["bbox"]).astype(np.int32)
                #img = cv2.rectangle(img, (x,y),(x + w,y + h),(0,255,255), 3) 
                img = draw_xywha(img,x,y,w,h,a)
                cat = coco.cats[anno['category_id']]['name']
                cv2.putText(img, cat, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX ,  0.8, 1, cv2.LINE_AA)            
                
    
    plt.imshow(img[...,::-1])
    plt.axis("off")
    plt.grid(False)
    plt.tight_layout()

def get_angles(ann):
    
    #print(ann['bbox'])
    img_id = ann['image_id']
    new_ann = None
    # get width and height
    if not 'rbbox' in ann:
        # using COCO dataset. 4 = [x1,y1,w,h]
        coco = True
        # convert COCO format: x1,y1,w,h to x,y,w,h
        ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
        ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
        ann['bbox'].append(0)
        if ann['bbox'][2] > ann['bbox'][3]:
            ann['bbox'][2],ann['bbox'][3] = ann['bbox'][3],ann['bbox'][2]
            ann['bbox'][4] -= 90
        new_ann = ann['bbox']
    else:
        # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
        assert len(ann['rbbox']) == 5, 'Unknown bbox format' # x,y,w,h,a
        new_ann = ann['rbbox']
        

        if new_ann[2] > new_ann[3]:
            new_ann[2], new_ann[3] = new_ann[3],new_ann[2]
            new_ann[4] -= 90 if new_ann[4] > 0 else -90

    if new_ann[2] == new_ann[3]:
        new_ann[3] += 1 # force that w < h
        
    new_ann[4] = np.clip(new_ann[4], -90.0, 90.0 - 1e-14)

    assert new_ann[2] < new_ann[3]
    assert new_ann[4] >= -90 and new_ann[4] < 90

    # override original bounding box with rotated bounding box
    ann['bbox'] = torch.Tensor(new_ann)
    #print(ann['bbox'])
    return ann

    
def draw_xywha(im, x, y, w, h, angle, color=(255,0,0), linewidth=5):
    '''
    im: image numpy array, shape(h,w,3), RGB
    angle: degree
    '''
    c, s = np.cos(angle/180*np.pi), np.sin(angle/180*np.pi)
    R = np.asarray([[c, s], [-s, c]])
    pts = np.asarray([[-w/2, -h/2], [w/2, -h/2], [w/2, h/2], [-w/2, h/2]])
    rot_pts = []
    for pt in pts:
        rot_pts.append(([x, y] + pt @ R).astype(int))
    contours = np.array([rot_pts[0], rot_pts[1], rot_pts[2], rot_pts[3]])
    cv2.polylines(im, [contours], isClosed=True, color=color,
                thickness=linewidth, lineType=cv2.LINE_4)
    return im
    
    
data_folder = "/localdata/saurabh/dataset/FES/JPEGImages/"
coco = COCO("/localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json")
#data_folder = "/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/JPEGImages/"
#coco = COCO("/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/instances_training.json")
plt.figure(figsize=(100,100))
total = len(coco.getImgIds()) - 1

interact(visualize, index=widgets.IntSlider(min=0, max=total, step=1, continuous_update=False), masks=False, bounding_boxes=True);  
plt.show()

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


<Figure size 7200x7200 with 0 Axes>

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=300), Checkbox(valu…

# Make the data files

In [1]:
######## NOTE #######
'''Last performed for FES dataset. The logic differes slightly for other datasets. 
Take care of image extensions and image_names variable during implementation.
Logic of load annotation is verified. 
Do visualize the targets before training to check if correct annotations are saved'''
####### NOTE ######



import glob
from collections import defaultdict
import os
import json
import numpy as np
import cv2
from PIL import Image
import torch
###Input####
mode_in = ''
dataset = ''
image_paths = glob.glob(f"/localdata/saurabh/yolov3/data/{dataset}/images/{mode_in}/*.jpg")
############


image_ids = []
image_names = []
#print(image_paths[0])
##### Edit this part according to the name of the files ####
for image in image_paths:
    #print(image)
    image_name, image_ext = os.path.splitext(os.path.basename(image))
    #print(image_name)
    image_names += [image_name]
    image_name = image_name.split('_')[1]     # Varies according to databases. Check if name is correct wrt to anno
    image_ids += [image_name]     # If image_ids != image_names, execute the function below to get sequential image ids
    #break
    
#print(image_ids)

In [2]:
len(image_ids)

200

In [3]:
image_names

['Record_00893',
 'Record_00888',
 'Record_00855',
 'Record_00826',
 'Record_00880',
 'Record_00886',
 'Record_00805',
 'Record_00803',
 'Record_00809',
 'Record_00812',
 'Record_00890',
 'Record_00858',
 'Record_00804',
 'Record_00879',
 'Record_00857',
 'Record_00881',
 'Record_00872',
 'Record_00852',
 'Record_00848',
 'Record_00802',
 'Record_00824',
 'Record_00842',
 'Record_00868',
 'Record_00818',
 'Record_00806',
 'Record_00836',
 'Record_00819',
 'Record_00816',
 'Record_00827',
 'Record_00887',
 'Record_00839',
 'Record_00837',
 'Record_00845',
 'Record_00873',
 'Record_00835',
 'Record_00877',
 'Record_00821',
 'Record_00831',
 'Record_00882',
 'Record_00823',
 'Record_00899',
 'Record_00889',
 'Record_00892',
 'Record_00900',
 'Record_00830',
 'Record_00870',
 'Record_00894',
 'Record_00844',
 'Record_00807',
 'Record_00897',
 'Record_00878',
 'Record_00859',
 'Record_00856',
 'Record_00832',
 'Record_00815',
 'Record_00884',
 'Record_00874',
 'Record_00863',
 'Record_00850

In [3]:
#### Process and load annotatinos according to RAPID implementation
def load_anns(img_dir,json_path):
        coco = False
        print(f'Loading annotations {json_path} into memory...')
        with open(json_path, 'r') as f:
            json_data = json.load(f)

        for ann in json_data['annotations']:
            img_id = ann['image_id']
            #print(img_id)
            new_ann = None
            # get width and height 
            if not 'rbbox' in ann:
                # using COCO dataset. 4 = [x1,y1,w,h]
                coco = True
                ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
                ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
                ann['bbox'].append(0)
                if ann['bbox'][2] > ann['bbox'][3]:
                    ann['bbox'][2],ann['bbox'][3] = ann['bbox'][3],ann['bbox'][2]
                    ann['bbox'][4] -= 90
                new_ann = ann['bbox']
            else:
                # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
                assert len(ann['rbbox']) == 5, 'Unknown bbox format' # x,y,w,h,a
                new_ann = ann['rbbox']


                if new_ann[2] > new_ann[3]:
                    new_ann[2], new_ann[3] = new_ann[3],new_ann[2]
                    new_ann[4] -= 90 if new_ann[4] > 0 else -90

            if new_ann[2] == new_ann[3]:
                new_ann[3] += 1 # force that w < h

            new_ann[4] = np.clip(new_ann[4], -90.0, 90.0 - 1e-14)

            assert new_ann[2] < new_ann[3]
            assert new_ann[4] >= -90 and new_ann[4] < 90

            # override original bounding box with rotated bounding box
            ann['bbox'] = torch.Tensor(new_ann)
            #print(img_id)
            imgid2anns[img_id].append(ann)

        for img in json_data['images']:
            img_id = img['id']
            #print(img_id)
            assert img_id not in imgid2path
            anns = imgid2anns[img_id]
            # if there is crowd gt, skip this image
            if coco and any(ann['iscrowd'] for ann in anns):
                continue

            img_ids.append(img_id)
            imgid2path[img_id] = os.path.join(img_dir, img['file_name'])
            imgid2info[img['id']] = img

        catids = [cat['id'] for cat in json_data['categories']]

In [4]:
### Store the required values generated during load_anns function
coco =  False
img_ids = []
imgid2info = dict()
imgid2path = dict()
imgid2anns = defaultdict(list)
catids = []

if dataset == 'fes':
    load_anns("/localdata/saurabh/dataset/FES/JPEGImages/", "/localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json" )
elif dataset == 'custom':
    load_anns("/localdata/saurabh/dataset/theodore_v3/images/", "/localdata/saurabh/dataset/theodore_v3/coco/annotations/instances.json")

Loading annotations /localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json into memory...


In [5]:
imgid2info[105] # just to cross check the image_id-image_name pair

{'id': 105,
 'width': 1680,
 'height': 1680,
 'file_name': 'Record_00704.jpg',
 'license': 0,
 'flickr_url': '',
 'coco_url': '',
 'date_captured': 0}

In [6]:
#### Run this to get image_ids in sequence starting with 1 #####
'''Do not run if image_id=image_name'''

image_ids = []
for i in range(len(imgid2info)):
    file_name = imgid2info[i+1]['file_name'].split('.')[0]
    #i_d = imgid2info
    
    if file_name in image_names:  # Make sure names in both list are identical
        image_ids += [i+1]
        print(file_name, i+1)
    

Record_00600 1
Record_00601 2
Record_00602 3
Record_00603 4
Record_00604 5
Record_00605 6
Record_00606 7
Record_00607 8
Record_00608 9
Record_00609 10
Record_00610 11
Record_00611 12
Record_00612 13
Record_00613 14
Record_00614 15
Record_00615 16
Record_00616 17
Record_00617 18
Record_00618 19
Record_00619 20
Record_00620 21
Record_00621 22
Record_00622 23
Record_00623 24
Record_00624 25
Record_00625 26
Record_00626 27
Record_00627 28
Record_00628 29
Record_00629 30
Record_00630 31
Record_00631 32
Record_00632 33
Record_00633 34
Record_00634 35
Record_00635 36
Record_00636 37
Record_00637 38
Record_00638 39
Record_00639 40
Record_00640 41
Record_00641 42
Record_00642 43
Record_00643 44
Record_00644 45
Record_00645 46
Record_00646 47
Record_00647 48
Record_00648 49
Record_00649 50
Record_00650 51
Record_00651 52
Record_00652 53
Record_00653 54
Record_00654 55
Record_00655 56
Record_00656 57
Record_00657 58
Record_00658 59
Record_00659 60
Record_00660 61
Record_00661 62
Record_00662 63
R

In [7]:
image_names = sorted(image_names) #if image_ids are sequential then sort the names accordingly

In [8]:
image_names

['Record_00600',
 'Record_00601',
 'Record_00602',
 'Record_00603',
 'Record_00604',
 'Record_00605',
 'Record_00606',
 'Record_00607',
 'Record_00608',
 'Record_00609',
 'Record_00610',
 'Record_00611',
 'Record_00612',
 'Record_00613',
 'Record_00614',
 'Record_00615',
 'Record_00616',
 'Record_00617',
 'Record_00618',
 'Record_00619',
 'Record_00620',
 'Record_00621',
 'Record_00622',
 'Record_00623',
 'Record_00624',
 'Record_00625',
 'Record_00626',
 'Record_00627',
 'Record_00628',
 'Record_00629',
 'Record_00630',
 'Record_00631',
 'Record_00632',
 'Record_00633',
 'Record_00634',
 'Record_00635',
 'Record_00636',
 'Record_00637',
 'Record_00638',
 'Record_00639',
 'Record_00640',
 'Record_00641',
 'Record_00642',
 'Record_00643',
 'Record_00644',
 'Record_00645',
 'Record_00646',
 'Record_00647',
 'Record_00648',
 'Record_00649',
 'Record_00650',
 'Record_00651',
 'Record_00652',
 'Record_00653',
 'Record_00654',
 'Record_00655',
 'Record_00656',
 'Record_00657',
 'Record_00658

In [9]:
#### Get annotations only for the train/val set according to image_ids
ind_path = [imgid2path[int(ids)] for ids in image_ids]
annotations = [imgid2anns[int(ids)] for ids in image_ids]

In [10]:
##### Map the labels to 0-indexed labels
label_map = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5}

In [11]:
label_map[3]

2

In [12]:
##### Save the paths to the images and annotations as .txt files

all_images = []
text_names = open(f'/localdata/saurabh/yolov3/data/{dataset}/{mode_in}_paths.txt', "w")
for i, ann in enumerate(annotations):
    file_name = image_names[i]     # image_names should be in sync with annotation sequence
    #### Append the path list
    text_names.write(f'/localdata/saurabh/yolov3/data/{dataset}/images/{mode_in}/{file_name}{image_ext}')
    text_names.write('\n')
    #### Open the label file for each anno
    text_file = open(f'/localdata/saurabh/yolov3/data/{dataset}/labels/{mode_in}/{file_name}.txt', "w") 
    for anno in ann:
        #print(anno['category_id'])
        text_file.write("%i " %label_map[anno['category_id']])
        text_file.writelines(["%f " % item  for item in anno['bbox']])
        text_file.write("\n")
    
    text_file.close()
    #break
text_names.close()

print(file_name)
    


Record_00799


In [ ]:
annotations

In [ ]:
with open('/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/captions_training.json', 'r') as f:
            json_data = json.load(f)

In [ ]:
json_data['images']